## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home") #On importe l'API et on se connecte sur le portail
"""Connexion successed"""

'Connexion successed'

In [2]:
import json #Pour manipuler le json
import urllib.request as urllib #Pour ouvrir une page internet
import pandas as pd
#import geojson #Pour JSON to GeoJSON Ne fonctionne pas, il faut télécharger le module sur \arcgis
from arcgis.mapping import WebMap #Pour la carte web

from arcgis.geometry import Point, Polyline, Polygon, Geometry #Pour dessiner les points
from arcgis.features import Feature #Pour Feature
from arcgis.features import FeatureSet
"""Modules importés"""

'Modules importés'

In [3]:
'''Récupération des données depuis le json hébergé sur le site data.gouv'''
req = urllib.Request("https://www.data.gouv.fr/fr/datasets/r/d0566522-604d-4af6-be44-a26eefa01756") #on nomme la requête
ouverture = urllib.urlopen(req) ##on ouvre le lien
data_gouv = json.loads(ouverture.read()) #le fichier json est dans la variable data_gouv sous la forme d'un dictionnaire

"""Requête effectuée et import des données fait"""

'Requête effectuée et import des données fait'

In [4]:
"""Création de la dataframe"""
df = pd.DataFrame(data_gouv["features"]) #presente sous la forme d'un tableau panda l'ensemble des données (features)
nbft = len(df) #Nombre d'entités a réutiliser dans la boucle pour créer les points
print('Il y a {} lignes'.format(nbft)) # On affiche le nombre de lignes
print("le nom des colonnes est :",str(df.columns.tolist())) #affiche le nom des colonnes de df
print("le type de df est: ",type(df))
"""Dataframe de longueur nbft créée"""

Il y a 2375 lignes
le nom des colonnes est : ['type', 'properties', 'geometry']
le type de df est:  <class 'pandas.core.frame.DataFrame'>


'Dataframe de longueur nbft créée'

In [5]:
"""Nomenclature des données de la df"""
typ = df['type']
#print(typ)
print("le type de type est :", type(typ),"\n")

###Properties
pro = df['properties']
print("le type de pro est:", type(pro))
print("Propriétés de la première entité :",pro[0],"\n")

###Geometry
geo = df['geometry']
#dfgeo = pd.DataFrame(geo)
print("le type de géo est:", type(geo),"\n") #Series c'est une liste
print(geo[0],"le type de geo[0] est :",type(geo[0]),"\n") #C'est un dictionnaire python
print(geo[0]['coordinates'],"le type de geo[0]['coordinates'] est :", type(geo[0]['coordinates']),"\n") #C'est une liste python

"""properties et geography enregistrées dans pro et geo"""

le type de type est : <class 'pandas.core.series.Series'> 

le type de pro est: <class 'pandas.core.series.Series'>
Propriétés de la première entité : {'gid': 1, 'c_gid': 15, 'c_nom': 'Grand centre de vaccination', 'c_xy_precis': '1', 'c_id_adr': '08105_4000_00022', 'c_adr_num': None, 'c_adr_voie': 'Rue de la Vieille Meuse', 'c_com_cp': '08000', 'c_com_insee': '08105', 'c_com_nom': 'Charleville-Mézières', 'c_lat_coor1': 49.7567, 'c_long_coor1': 4.71338, 'c_structure_siren': '130007834', 'c_structure_type': '8412Z', 'c_structure_rais': 'AGENCE REGIONALE DE SANTE GRAND EST', 'c_structure_num': '3', 'c_structure_voie': 'BD JOFFRE', 'c_structure_cp': '54000', 'c_structure_insee': '54395', 'c_structure_com': 'NANCY', 'c__edit_datemaj': '2021/03/26 11:16:05.332', 'c_lieu_accessibilite': None, 'c_rdv_lundi': '9:00-13:00/14:00-18:00', 'c_rdv_mardi': '9:00-13:00/14:00-18:00', 'c_rdv_mercredi': '9:00-13:00/14:00-18:00', 'c_rdv_jeudi': '9:00-13:00/14:00-18:00', 'c_rdv_vendredi': '9:00-13:00/14:00

'properties et geography enregistrées dans pro et geo'

In [6]:
"""Ouverture de la couche à modifier"""
#access the item's feature layers
items = gis.content.search("title:Couche Donnees Centres Vaccination Gouv")
item = items[0]
mylayer = item.layers[0]
type(mylayer)
#mylayer.properties.capabilities
"""Couche d'entité ouverte"""

"Couche d'entité ouverte"

In [7]:
"""Suppression des anciens points"""
#mylayer.manager.truncate()
mylayer.delete_features(where="objectid > 0")
"""La carte a été nettoyée"""

'La carte a été nettoyée'

In [8]:
"""Création des points"""
###Après avoir nettoyé le feature layer on peut recréer les points
###Attention pour que les attributs s'affiches on a d'abord créé les champs manuellement
###Attention à la taille des champs exemple pour le gid 347 de cgid 491 qui a bcp de caractere en rdv modalité
### On ne peut donc pas inclure les modalités de rdv tant que cet item n'est pas modifié

#for i in range(nbft): #on va faire 0 jusqu'à nbft -1

centres = []
for i in range(nbft): #On fait tourner le code avec toutes les lignes de la dataframe
#for i in range(300,370): ###Tester avec quelques valeurs
    coord = geo[i]['coordinates'][0]# On affiche les i premières coordonnées des centres
    x = coord[0]
    y = coord[1]
    pt = Point({"x" : x, "y" : y, "spatialReference" : {"wkid" : 4326}})# on crée le point #En mettant 3857 on a pas du tout les bons points 
    #attribut1 = {"nom" : "c_nom","gid" : "100","tag":"test"} #On nomme les attributs ligne à remplacer par properties
    attribut = pro[i] #Si les champs existent sur le layers ils vont être remplis par les données de pro[i]
    ft = Feature(attributes = attribut, geometry = pt) #On crée l'entité avec la fonction feature qui emporte la localisation et les attributs
    #display(ft) #affiche les features nouvellement créés 1 par 1
    centres.append(ft) #On ajoute l'entité à la liste

#print(centres[:2])
#print(len(centres))
    
# Ne pas tenir compte des problème d'encoding dans les lignes suivantes
"""Les points ont été créés"""

'Les points ont été créés'

In [9]:
"""Publication de la data"""
add_result = mylayer.edit_features(adds = centres)
display(add_result)
#centres_fset = FeatureSet(features = features, geometry_type="Point", spatial_reference={'latestWkid': 4326, 'wkid': 102100})
#{"wkid": 102100, "latestWkid": 3857}}

{'addResults': [{'objectId': 250270,
   'uniqueId': 250270,
   'globalId': 'F7406E6F-1B8E-4615-97FE-9841C1068700',
   'success': True},
  {'objectId': 250271,
   'uniqueId': 250271,
   'globalId': 'DAC3D49A-2D01-4898-ABB3-EABA2C9E9196',
   'success': True},
  {'objectId': 250272,
   'uniqueId': 250272,
   'globalId': '1D869ECA-5612-48FB-AB94-55C89DD245D6',
   'success': True},
  {'objectId': 250273,
   'uniqueId': 250273,
   'globalId': '3F4AF112-DE82-41F9-8378-329DEED05FF5',
   'success': True},
  {'objectId': 250274,
   'uniqueId': 250274,
   'globalId': '99CE3826-827F-4C0F-B940-84E0E8713974',
   'success': True},
  {'objectId': 250275,
   'uniqueId': 250275,
   'globalId': 'F78A9A46-080F-4F5A-A1E8-24DE6C17AB49',
   'success': True},
  {'objectId': 250276,
   'uniqueId': 250276,
   'globalId': 'A9BBEC63-8599-47E6-A50C-79A2CE4F5590',
   'success': True},
  {'objectId': 250277,
   'uniqueId': 250277,
   'globalId': 'CCD6AA83-1BDF-4793-BB91-9F807C570891',
   'success': True},
  {'object

In [10]:
"""Affichage de la table pour vérification"""
fset_edited = mylayer.query()
fset_edited.sdf

,OBJECTID,GlobalID,CreationDate,Creator,EditDate,Editor,c_nom,c_rdv_lundi,gid,c_xy_precis,...,c_gid,c_adr_num,c_adr_voie,c_com_cp,c_com_insee,c_com_nom,c_lieu_accessibilite,c_rdv_site_web,c_centre_fermeture,SHAPE
0,250270,f7406e6f-1b8e-4615-97fe-9841c1068700,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,Grand centre de vaccination,9:00-13:00/14:00-18:00,1,1.000000,...,15,None,Rue de la Vieille Meuse,08000,08105,Charleville-Mézières,None,https://partners.doctolib.fr/hopital-public/ch...,0,"{""x"": 524690.8388762182, ""y"": 6404241.70157159..."
1,250271,dac3d49a-2d01-4898-abb3-eaba2c9e9196,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,CHU DE CAEN NORMANDIE,None,2,0.999981,...,17,None,avenue Côte de Nacre,14000,14118,Caen,None,https://partners.doctolib.fr/hopital-public/ca...,None,"{""x"": -39783.54680820393, ""y"": 6309637.3034826..."
2,250272,1d869eca-5612-48fb-ab94-55c89dd245d6,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,Centre de vaccination - Centre des congrès de ...,None,3,0.999999,...,18,16,Rue de la Société Française,18100,18279,Vierzon,None,https://partners.doctolib.fr/centre-de-sante/v...,None,"{""x"": 229356.55322198317, ""y"": 5978812.2790342..."
3,250273,3f4af112-de82-41f9-8378-329deed05ff5,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,Centre de vaccination de Saint-Lô : Salle Allende,None,4,1.000000,...,19,2,Rue bis Saint-Thomas - salle salvador allende,50000,50502,Saint-Lô,None,https://partners.doctolib.fr/centre-de-sante/s...,0,"{""x"": -121506.62484459892, ""y"": 6294197.623477..."
4,250274,99ce3826-827f-4c0f-b940-84e0e8713974,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,CH Bastia,8:00-17:00,5,1.000000,...,21,604,Chemin de Falconaja,20600,2B033,Bastia,OUI,https://partners.doctolib.fr/hopital-public/ba...,None,"{""x"": 1050376.0399580016, ""y"": 5263524.1021233..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,252640,3247467c-dee4-4bb6-bfc5-c6688aedcd72,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,"Centre de vaccination - commune de Caudry, sal...",fermé,2371,1.000000,...,2646,None,Rue Ampère,59540,59139,Caudry,None,https://www.doctolib.fr/centre-de-sante/caudry...,None,"{""x"": 378759.89200550003, ""y"": 6468802.6986809..."
2371,252641,d2922b5b-68f5-4118-b1fc-88720c6cedb2,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,Centre de vaccination - Mairie d'Onnaing,fermé,2372,1.000000,...,2647,270,Rue Jean Jaurès,59264,59447,Onnaing,None,None,None,"{""x"": 400810.0567418316, ""y"": 6513475.38841938..."
2372,252642,4874725a-3322-494f-a093-d45e229d2ae2,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,CDV31 Villeneuve de Rivière,9h-19h,2373,0.999991,...,2648,1,route de la croix de Cassagne,31800,31585,Villeneuve-de-Rivière,OUI,https://www.keldoc.com/cabinet-medical/villene...,1,"{""x"": 77495.50107533028, ""y"": 5329453.73277930..."
2373,252643,66049cf3-fe09-4167-8740-e8af3cff368f,2021-05-31 09:00:54.367000103,mdein_mapthenews,2021-05-31 09:00:54.367000103,mdein_mapthenews,CDV31 Montastruc La Conseillère,9h-19h,2374,0.999999,...,2649,5,Rue Rene Delmas,31380,31358,Montastruc-la-Conseillère,OUI,https://www.keldoc.com/cabinet-medical/montast...,0,"{""x"": 176108.5572816228, ""y"": 5423112.95483109..."
